In [ ]:
import argparse
import sys
import torch
import os

In [ ]:
def main():
    # Parse command line arguments
    parser = argparse.ArgumentParser(description='Translate using a pre-trained model')
    parser.add_argument('--model', help='a model previously trained with train.py')
    parser.add_argument('--batch_size', type=int, default=50, help='the batch size (defaults to 50)')
    parser.add_argument('--beam_size', type=int, default=12, help='the beam size (defaults to 12, 0 for greedy search)')
    parser.add_argument('--encoding', default='utf-8', help='the character encoding for input/output (defaults to utf-8)')
    parser.add_argument('-i', '--input', default=sys.stdin.fileno(), help='the input folder (defaults to stdin)')
    parser.add_argument('-o', '--output', default=sys.stdout.fileno(), help='the output folder (defaults to stdout)')
    args = parser.parse_args(args=[])

    args.batch_size=128
    args.model="model/arm32_to_x86.it22000.src2trg.pth"
    args.input="data/arm32_malware"
    args.output="data/arm32_malware_translate"

    # Load model
    translator = torch.load(args.model)

    # Create the output folder if it doesn't exist
    os.makedirs(args.output, exist_ok=True)

    # Translate sentences for each file in the input folder
    for filename in os.listdir(args.input):
        input_file = os.path.join(args.input, filename)
        output_file = os.path.join(args.output, filename)

        with open(input_file, 'r', encoding=args.encoding, errors='surrogateescape') as fin, \
             open(output_file, 'w', encoding=args.encoding, errors='surrogateescape') as fout:

            end = False
            while not end:
                batch = []
                while len(batch) < args.batch_size and not end:
                    line = fin.readline()
                    if not line:
                        end = True
                    else:
                        batch.append(line)
                if args.beam_size <= 0 and len(batch) > 0:
                    for translation in translator.greedy(batch, train=False):
                        print(translation, file=fout)
                elif len(batch) > 0:
                    for translation in translator.beam_search(batch, train=False, beam_size=args.beam_size):
                        print(translation, file=fout)
                fout.flush()

if __name__ == '__main__':
    main()